In [3]:
%load_ext autoreload
%autoreload 2
import os 
os.chdir("/Users/luohy/Documents/Projects/bus-observatory/gtfs-realtime-performance")
from src.s3 import list_files_in_bucket, filter_files_by_pattern, read_parquet_from_s3
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.gtfs_segments import GTFS_shape_processor
from src.speeds import BusSpeedCalculator
from src.api import parse_zipped_gtfs
import geopandas as gpd
from src.api import query_feed_data, get_access_token
ACCESS_TOKEN = get_access_token()
pd.set_option('display.max_columns', None)

In [4]:
prefix = "feeds/nyct_mta_bus_gtfsrt/"

In [5]:
def load_all_parquet_files(file_list, bucket="busobservatory-lake", max_workers=100):
    dfs = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(read_parquet_from_s3, bucket, key) for key in file_list]
        for future in as_completed(futures):
            try:
                dfs.append(future.result())
            except Exception as e:
                print(f"Error reading a file: {e}")
    return pd.concat(dfs, ignore_index=True)

# Load Relevant realtime data

In [8]:
daily_files = list_files_in_bucket("busobservatory-lake", prefix = prefix)
daily_files_1 = [daily_files[0]]
print("Checking parquet file:", daily_files_1)
vehicle_positions_1 = load_all_parquet_files(daily_files_1)

Checking parquet file: ['feeds/nyct_mta_bus_gtfsrt/COMPACTED_nyct_mta_bus_gtfsrt_2023-04-28_13:41:56.parquet']


In [11]:
print(vehicle_positions_1.columns)
print(vehicle_positions_1.shape)
print(vehicle_positions_1.head())

Index(['id', 'vehicle.trip.trip_id', 'vehicle.trip.start_date',
       'vehicle.trip.route_id', 'vehicle.trip.direction_id',
       'vehicle.position.latitude', 'vehicle.position.longitude',
       'vehicle.position.bearing', 'vehicle.timestamp', 'vehicle.stop_id',
       'vehicle.vehicle.id'],
      dtype='object')
(1204598, 11)
              id                vehicle.trip.trip_id vehicle.trip.start_date  \
0  MTA NYCT_5331  OH_B3-Weekday-SDon-116200_M101_143                20230427   
1  MTA NYCT_5330  OH_B3-Weekday-SDon-114600_M101_128                20230427   
2  MTA NYCT_5335   KB_B3-Weekday-SDon-113400_BX1_130                20230427   
3  MTA NYCT_5334   KB_B3-Weekday-SDon-117000_BX2_221                20230427   
4  MTA NYCT_7989    UP_B3-Weekday-SDon-117800_B6_316                20230427   

  vehicle.trip.route_id  vehicle.trip.direction_id  vehicle.position.latitude  \
0                  M102                        1.0                  40.791969   
1                  M101  